In [2]:
import pandas as pd

In [6]:
#df = pd.read_csv('final_book3.csv')

In [3]:
df = pd.read_csv('emb_value.csv')

import ast
# 문자열을 리스트로 변환
df['embedding_책소개'] = df['embedding_책소개'].apply(lambda x: list(map(float, x.replace('\n', '')[1:-1].split())))

from gensim.models import FastText

In [7]:
# FastText 모델 불러오기
fasttext_model = FastText.load(r'C:\Users\mysorol\바탕 화면\imworking\chkchk\contents_based\fasttext_model.bin')

UnpicklingError: could not find MARK

In [37]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력받기
user_books = ['바보멍청이똥개','트렌드 코리아 2025', '회계 천재가 된 홍대리', '모두의 금리', '돈 공부를 시작하고 인생의 불안이 사라졌다', '도쿄 트렌드 인사이트 2025', '레버리지', '부의 시나리오', '로지컬 씽킹', '피터 드러커 자기경영노트', '60년대생이 온다','자본주의']
M = 10  # 추천할 책 개수인사

# 1. DB에서 일치하는 책 찾기 (제목 컬럼 사용)
matched_books = df[df['title'].isin(user_books)]

if matched_books.shape[0] == 0:
    print("일치하는 책이 없습니다.")
else:
    # 2. 입력한 책들의 책소개 임베딩 가져오기
    matched_embeddings = np.array([
        df['embedding_책소개'].iloc[i]
        for i in matched_books.index
    ])

    # 3. 모든 책의 책소개 임베딩 결합
    all_embeddings = np.array([
        df['embedding_책소개'].iloc[i]
        for i in range(len(df))
    ])

    # 4. 사용자 책 임베딩 평균
    user_embedding = np.mean(matched_embeddings, axis=0).reshape(1, -1)

    # 5. 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_embedding, all_embeddings).flatten()

    # 6. 가장 유사한 M권의 책 추천 (단, 입력한 책과 일치하는 책은 제외)
    recommended_books_indices = np.argsort(similarity_scores)[-M-1:-1]  # 가장 유사한 M권 인덱스
    # recommended_books_indices = np.argpartition(similarity_scores, -M-1)[-M-1:]
    recommended_books_indices = [idx for idx in recommended_books_indices if idx not in matched_books.index]  # 제외


    # 7. 추천 책 가져오기
    recommended_books = df.iloc[recommended_books_indices]
    recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])
    recommended_books[['title', 'full_name', 'cos']].sort_values(by='cos', ascending=False)

C:\Users\nunic\AppData\Local\Temp\ipykernel_19416\857311142.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])


In [39]:
recommended_books[['title', 'full_name','cos']].sort_values(by='cos', ascending=False)

,title,full_name,cos
24094,2015 현대경제연구원 휴가철 선정도서 세트,2015 현대경제연구원 휴가철 선정도서 세트 [ 전11권 ],0.923334
11688,골든 크로스,골든 크로스 주식과 부동산의 위기를 기회로 만드는 투자 전략,0.920871
5830,100대 기업 ESG 담당자가 가장 자주 하는 질문,100대 기업 ESG 담당자가 가장 자주 하는 질문,0.918648
17110,그린란드상어처럼 생존하라,그린란드상어처럼 생존하라 부분에선 실패해도 전체에선 이기는 기업의 생존기술,0.916318
51856,내 인생의 후반전은 아직 결정되지 않았다,내 인생의 후반전은 아직 결정되지 않았다,0.916292
12531,부의 재편,"부의 재편 새로운 부와 마켓, 그리고 전혀 다른 기회",0.916143
4604,장하준의 경제학 명강의 시리즈 세트,장하준의 경제학 명강의 시리즈 세트 나쁜 사마리아인들 + 그들이 말하지 않는 23가...,0.916015
37463,후즈유어시티 WHO'S YOUR CITY,후즈유어시티 WHO'S YOUR CITY 세계의 경제 엘리트들은 어디서 사는가,0.915794
20948,퍼스트 무버 4차 산업혁명의 선도자들,퍼스트 무버 4차 산업혁명의 선도자들,0.915412
1258,피크아웃 코리아,피크아웃 코리아 미래가 없는 사회에서 살아남기,0.915330


In [19]:
x = df['embedding_책소개'][3]
x

'[ 0.04092156  1.1133122   0.09052531 -0.39455134  0.04127113 -0.266976\n -0.3475721   0.17435507 -0.3709432  -0.71702355 -0.3044962   0.4867803\n  0.01164729 -0.10703948 -0.80601984  0.8737641  -0.3737457   0.50404596\n  0.16038355  0.7477061  -0.3160667  -0.9024943  -0.70185596  0.7842009\n -0.04970044 -0.09249415  0.15315756  0.5105449   0.18726115 -0.3059648\n  0.00770184 -0.6144606  -1.1997863  -0.37914553 -1.2983112   0.26560637\n  0.9054487   0.30595112 -0.1032512  -0.21743514  0.020437   -0.15651014\n -0.14259472 -0.7517462   0.02914969 -0.39765593  0.34780216 -0.34752834\n -0.9567127   0.19733387 -1.0115819  -0.31952977 -0.12017702 -0.0381263\n  0.08930171 -0.3932289  -0.37033647  0.49290457 -0.12076207 -0.17082429\n  0.39826146 -1.3247088  -0.42003566  0.36601558  0.0137801  -0.31784308\n -0.00620803 -0.5367416  -0.29305542  0.02867975  0.07108983  0.5972097\n  0.29905984  0.00325138  0.4339783   0.5860185   0.34903544 -0.3006662\n -0.16366321  0.18403576 -1.2186208  -0.48975

In [26]:
import ast
# 문자열을 리스트로 변환
df['embedding_책소개'] = df['embedding_책소개'].apply(lambda x: list(map(float, x.replace('\n', '')[1:-1].split())))

AttributeError: 'list' object has no attribute 'replace'

In [41]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 입력받기
user_books = ['바보멍청이똥개', '트렌드 코리아 2025', '회계 천재가 된 홍대리', '모두의 금리', '돈 공부를 시작하고 인생의 불안이 사라졌다', 
              '도쿄 트렌드 이트 2025', '레버리지', '부의 시나리오', '로지컬 씽킹', '피터 드러커 자기경영노트', '60년대생이 온다', '자본주의']
M = 10  # 추천할 책 개수

# 1. DB에서 일치하는 책 찾기 (제목 컬럼 사용)
matched_books = df[df['title'].isin(user_books)]

# 2. 존재하지 않는 책 찾기
missing_books = set(user_books) - set(matched_books['title'])

if missing_books:
    print(f"다음 책들은 DB에 없습니다: {', '.join(missing_books)}")

# 존재하는 책이 있는 경우에만 추천 로직 진행
if not matched_books.empty:
    # 3. 입력한 책들의 책소개 임베딩 가져오기
    matched_embeddings = np.array([
        df['embedding_책소개'].iloc[i]
        for i in matched_books.index
    ])

    # 4. 모든 책의 책소개 임베딩 결합
    all_embeddings = np.array([
        df['embedding_책소개'].iloc[i]
        for i in range(len(df))
    ])

    # 5. 사용자 책 임베딩 평균
    user_embedding = np.mean(matched_embeddings, axis=0).reshape(1, -1)

    # 6. 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_embedding, all_embeddings).flatten()

    # 7. 가장 유사한 M권의 책 추천 (단, 입력한 책과 일치하는 책은 제외)
    recommended_books_indices = np.argsort(similarity_scores)[-M-1:-1]  # 가장 유사한 M권 인덱스
    recommended_books_indices = [idx for idx in recommended_books_indices if idx not in matched_books.index]  # 입력한 책 제외

    # 8. 추천 책 가져오기
    recommended_books = df.iloc[recommended_books_indices]
    recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])
    recommended_books = recommended_books[['title', 'full_name', 'cos']].sort_values(by='cos', ascending=False)

    # 추천 책 출력
    print(recommended_books)
else:
    print("입력된 책 중 일치하는 책이 없습니다.")

다음 책들은 DB에 없습니다: 도쿄 트렌드 이트 2025, 바보멍청이똥개
                              title  \
11688                        골든 크로스   
24094      2015 현대경제연구원 휴가철 선정도서 세트   
3780                 비열한 시장과 도마뱀의 뇌   
5830   100대 기업 ESG 담당자가 가장 자주 하는 질문   
4604            장하준의 경제학 명강의 시리즈 세트   
9555              미래의 부 + 미국주식 처음공부   
8741              책임지는 경영자 정의로운 투자자   
40556              월급쟁이 부자들 + 똑똑한 돈   
37463        후즈유어시티 WHO'S YOUR CITY   
9543                    미래의 부 + 에이트   

                                               full_name       cos  
11688                  골든 크로스 주식과 부동산의 위기를 기회로 만드는 투자 전략  0.916235  
24094                2015 현대경제연구원 휴가철 선정도서 세트   [ 전11권 ]  0.915509  
3780     비열한 시장과 도마뱀의 뇌 경제학과 뇌과학이 밝혀낸 초수익을 내는 비상식적 투자 법칙  0.913046  
5830                        100대 기업 ESG 담당자가 가장 자주 하는 질문  0.912847  
4604   장하준의 경제학 명강의 시리즈 세트 나쁜 사마리아인들 + 그들이 말하지 않는 23가...  0.911815  
9555                                   미래의 부 + 미국주식 처음공부  0.908858  
8741                   책임지는 경영자 정의로운 투자자 ESG로 

C:\Users\nunic\AppData\Local\Temp\ipykernel_19416\405610143.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])


In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt  # Okt 형태소 분석기 불러오기
from gensim.models import FastText


# FastText 모델 불러오기 (모델 경로에 맞게 수정 필요)
fasttext_model = FastText.load('C:/Users/nunic/Desktop/workspace/gamnyam_package/study/project_PJT/3_책쳌/web/streamlit_/fasttext_model.bin')

# Okt 형태소 분석기 불러오기
okt = Okt()

# 사용자 입력받기
user_books = ['트렌드 코리아 2025', '회계 천재가 된 홍대리', '모두의 금리', '돈 공부를 시작하고 인생의 불안이 사라졌다', 
              '도쿄 트렌드 인사이트 2025', '레버리지', '부의 시나리오', '로지컬 씽킹', '피터 드러커 자기경영노트', '60년대생이 온다', '자본주의']
M = 10  # 추천할 책 개수

# 1. DB에서 일치하는 책 찾기 (제목 컬럼 사용)
matched_books = df[df['title'].isin(user_books)]

# 2. 존재하지 않는 책 찾기
missing_books = set(user_books) - set(matched_books['title'])

if missing_books:
    print(f"다음 책들은 DB에 없습니다: {', '.join(missing_books)}")

# 존재하는 책들의 임베딩 가져오기
matched_embeddings = np.array([
    df['embedding_책소개'].iloc[i]
    for i in matched_books.index
])

# 3. 존재하지 않는 책의 제목을 Okt로 토큰화하고 임베딩 계산
missing_embeddings = []
if missing_books:
    for book in missing_books:
        tokens = okt.morphs(book)  # Okt를 이용한 토큰화
        embedding = np.mean([fasttext_model.wv[token] for token in tokens if token in fasttext_model.wv], axis=0)  # FastText 임베딩 계산
        missing_embeddings.append(embedding)

# 4. 존재하는 책의 임베딩과 없는 책의 임베딩을 합쳐서 평균 계산
if matched_embeddings.size > 0 or missing_embeddings:
    total_embeddings = np.vstack([matched_embeddings] + missing_embeddings)  # 임베딩 결합
    user_embedding = np.mean(total_embeddings, axis=0).reshape(1, -1)  # 임베딩 평균 계산

    # 5. 모든 책의 책소개 임베딩 결합
    all_embeddings = np.array([
        df['embedding_책소개'].iloc[i]
        for i in range(len(df))
    ])

    # 6. 코사인 유사도 계산
    similarity_scores = cosine_similarity(user_embedding, all_embeddings).flatten()

    # 7. 가장 유사한 M권의 책 추천 (단, 입력한 책과 일치하는 책은 제외)
    recommended_books_indices = np.argsort(similarity_scores)[-M-1:-1]  # 가장 유사한 M권 인덱스
    recommended_books_indices = [idx for idx in recommended_books_indices if idx not in matched_books.index]  # 입력한 책 제외

    # 8. 추천 책 가져오기
    recommended_books = df.iloc[recommended_books_indices]
    recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])
    recommended_books = recommended_books[['title', 'full_name', 'cos']].sort_values(by='cos', ascending=False)

    # 추천 책 출력
    print(recommended_books)
else:
    print("입력된 책 중 일치하는 책이 없습니다.")


                              title  \
24094      2015 현대경제연구원 휴가철 선정도서 세트   
11688                        골든 크로스   
5830   100대 기업 ESG 담당자가 가장 자주 하는 질문   
17110                 그린란드상어처럼 생존하라   
51856        내 인생의 후반전은 아직 결정되지 않았다   
12531                         부의 재편   
4604            장하준의 경제학 명강의 시리즈 세트   
37463        후즈유어시티 WHO'S YOUR CITY   
20948          퍼스트 무버 4차 산업혁명의 선도자들   
1258                       피크아웃 코리아   

                                               full_name       cos  
24094                2015 현대경제연구원 휴가철 선정도서 세트   [ 전11권 ]  0.923334  
11688                  골든 크로스 주식과 부동산의 위기를 기회로 만드는 투자 전략  0.920871  
5830                        100대 기업 ESG 담당자가 가장 자주 하는 질문  0.918648  
17110          그린란드상어처럼 생존하라 부분에선 실패해도 전체에선 이기는 기업의 생존기술  0.916318  
51856                             내 인생의 후반전은 아직 결정되지 않았다  0.916292  
12531                      부의 재편 새로운 부와 마켓, 그리고 전혀 다른 기회  0.916143  
4604   장하준의 경제학 명강의 시리즈 세트 나쁜 사마리아인들 + 그들이 말하지 않는 23가...  0.916015  
37463        후즈유어시

C:\Users\nunic\AppData\Local\Temp\ipykernel_19416\1949704286.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommended_books['cos'] = recommended_books.index.map(lambda x: similarity_scores[x])
